### IMPORTING THE REQUIRED MODULES

In [ ]:
!pip install transformers 
!pip install datasets
!pip install pynvml
!pip install evaluate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [ ]:
import transformers
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import os
import nltk
import torch
import evaluate
import sys
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### UTILS FUNCTION TO WORK WITH GPU

In [ ]:
# define utils functions to facilitate gpu 

def check_gpu_availability():
    # Check if CUDA is available
    print(f"Cuda is available: {torch.cuda.is_available()}")

def getting_device(gpu_prefence=True) -> torch.device:
    """
    This function gets the torch device to be used for computations, 
    based on the GPU preference specified by the user.
    """
    
    # If GPU is preferred and available, set device to CUDA
    if gpu_prefence and torch.cuda.is_available():
        device = torch.device('cuda')
    # If GPU is not preferred or not available, set device to CPU
    else: 
        device = torch.device("cpu")
    
    # Print the selected device
    print(f"Selected device: {device}")
    
    # Return the device
    return device

# Define a function to print GPU memory utilization
def print_gpu_utilization():
    # Initialize the PyNVML library
    nvmlInit()
    # Get a handle to the first GPU in the system
    handle = nvmlDeviceGetHandleByIndex(0)
    # Get information about the memory usage on the GPU
    info = nvmlDeviceGetMemoryInfo(handle)
    # Print the GPU memory usage in MB
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

# Define a function to print training summary information
def print_summary(result):
    # Print the total training time in seconds
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    # Print the number of training samples processed per second
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    # Print the GPU memory utilization
    print_gpu_utilization()

In [ ]:
# CHECK IF GPU IS UP
check_gpu_availability()

Cuda is available: True


In [ ]:
# SAVE THE DEVICE WE ARE WORKING WITH
device = getting_device(gpu_prefence=True)

Selected device: cuda


In [ ]:
# SHOULD BE FEW MB
print_gpu_utilization()

GPU memory occupied: 261 MB.


### IMPORTING THE DATA

In [ ]:
# Read in train and test CSV files using Pandas
path2train = '/content/drive/MyDrive/LT_SHARED_FOLDER/train.csv'
df = pd.read_csv(path2train)
# split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
train_df.head(n=10)

,id,keyword,location,text,target
4996,7128,military,Texas,Courageous and honest analysis of need to use ...,1
3263,4688,engulfed,NaN,@ZachZaidman @670TheScore wld b a shame if tha...,0
4907,6984,massacre,Cottonwood Arizona,Tell @BarackObama to rescind medals of 'honor'...,1
2855,4103,drought,"Spokane, WA",Worried about how the CA drought might affect ...,1
4716,6706,lava,"Medan,Indonesia",@YoungHeroesID Lava Blast &amp; Power Red #Pan...,0
7538,10777,wreckage,NaN,Wreckage 'Conclusively Confirmed' as From MH37...,1
3172,4551,emergency%20plan,Ireland,Our builder is having a dental emergency. Whic...,1
3932,5590,flood,NaN,BMX issues Areal Flood Advisory for Shelby [AL...,1
5833,8334,rubble,Made Here In Detroit,#360WiseNews : China's Stock Market Crash: Are...,1
7173,10278,war%20zone,NaN,@RobertONeill31 Getting hit by a foul ball whi...,0


In [ ]:
train_df.rename(columns = {"target":"labels"}, inplace = True)
test_df.rename(columns = {"target":"labels"}, inplace = True)
train_df.head(10)

,id,keyword,location,text,labels
4996,7128,military,Texas,Courageous and honest analysis of need to use ...,1
3263,4688,engulfed,NaN,@ZachZaidman @670TheScore wld b a shame if tha...,0
4907,6984,massacre,Cottonwood Arizona,Tell @BarackObama to rescind medals of 'honor'...,1
2855,4103,drought,"Spokane, WA",Worried about how the CA drought might affect ...,1
4716,6706,lava,"Medan,Indonesia",@YoungHeroesID Lava Blast &amp; Power Red #Pan...,0
7538,10777,wreckage,NaN,Wreckage 'Conclusively Confirmed' as From MH37...,1
3172,4551,emergency%20plan,Ireland,Our builder is having a dental emergency. Whic...,1
3932,5590,flood,NaN,BMX issues Areal Flood Advisory for Shelby [AL...,1
5833,8334,rubble,Made Here In Detroit,#360WiseNews : China's Stock Market Crash: Are...,1
7173,10278,war%20zone,NaN,@RobertONeill31 Getting hit by a foul ball whi...,0


In [ ]:
len(train_df), len(test_df)

(6090, 1523)

### BUILDING IT INTO DATASETES

In [ ]:
# pandas2dataset
ds_train = Dataset.from_pandas(train_df)
ds_test = Dataset.from_pandas(test_df)

In [ ]:
ds_train

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'labels', '__index_level_0__'],
    num_rows: 6090
})

### MODEL CHOICE

In [ ]:
model_nm = "bert-large-uncased" 

### TOKENIZING

In [ ]:
# IMPORTING THE MODULE TO GET THE TOKENIZER
from transformers import AutoTokenizer

In [ ]:
# IMPORTING OUR TOKENIZER
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
# DEFINING A TOKENIZE FUNCTION TO TOKENIZE BOTH THE TWO DATASETS
def tok_func(x): return tokz(x["text"], truncation=True, padding = "max_length")

In [ ]:
# CHECK THAT TOKENIZER FUNCTION WORKS
tok_func(ds_train[19]) # the 1 are for padding it; the attention mask show to not care about the 1

{'input_ids': [101, 5003, 12502, 14643, 1024, 22939, 17822, 8067, 1024, 2416, 11837, 3401, 4402, 1024, 10556, 2854, 5244, 4801, 2697, 2003, 1037, 11351, 2697, 2284, 1999, 1996, 10556, 2854, 5244, 4801, 12779, 6679, 16414, 7946, 8464, 1012, 2007, 8740, 1035, 8299, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 1018, 2080, 21472, 2692, 16715, 2620, 7295, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# TOKENIZING THE DS
tok_ds_train = ds_train.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])
tok_ds_test = ds_test.map(tok_func, batched=True, remove_columns=['text','id', 'keyword', 'location'])

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
# CREATE A DATASET TO FEED THE MODEL
ds = DatasetDict({"train":tok_ds_train,
             "test": tok_ds_test})

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [ ]:
# CHECK IF THE TOKENIZER WORKS, lets print the normal text
ds_train[19];

In [ ]:
# lets print the tokenized text
tok_ds_train[19];

In [ ]:
# let's convert the tokens into text and check if it is the same as the printed above
tokz.decode(ds["train"][19]["input_ids"])


'[CLS] maailiss : diaporama : sixpenceee : karymsky lake is a crater lake located in the karymsky volcanoaeinaerussia. with au _ http : / / t. co / 4o460fm8hn [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokz)

### IMPORTING THE MODEL

In [ ]:
# The reason behind this message is that 
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels = 2, use_cache = False).to(device) 

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
# checking if the model is on the gpu
print_gpu_utilization()

GPU memory occupied: 2119 MB.


### DEFINING METRICS

If we use the Trainer API to fine tune our model, we must define the metric function and pass it as one of the trainer argument; the trainer does not compute the metrics in automatic. Given the lack of class imbalance and cost imbalance, we will deploy Accuracy as evaluation strategy

In [ ]:
import numpy as np 
import evaluate 

In [ ]:
# we create an object called metrics; if the method "compute" is called and we pass in a list of predictions and true labels, it automatically compute them
metrics = evaluate.load("accuracy")

In [ ]:
# now that we have our callable object, we define a function that the trainer can use to compute its metric => we cannot call directly metrics.compute because the output
# of the model is not a prediction but a logist
def compute_accuracy(eval):
  
  # instantiating the two variables 
  logists, labels = eval 

  # transforming logists in prediction by picking; -1 applies argmax along the last axis of the input array 
  predictions = np.argmax(logists, axis = -1)    

  # computing the accuracy
  accuracy_score = metrics.compute(predictions=predictions, references=labels)

  return accuracy_score

### SETTING HYPERPARAMETERS OF BERT

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
# setting the hyperparameter for the trainer 
training_args = TrainingArguments(
    model_nm,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01, 
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8, # FROM BELOW MEMORY TRICKS
    gradient_accumulation_steps=16, # adding them to offset small batch size due to memory problem => so 2*8 => 16 batch-size traning
    gradient_checkpointing=True,
    fp16 = True
    )

### CREATING THE TRAINER

In [ ]:
# passing in the hyperparameter for the trainer 
trainer = Trainer(
    model = model, # our model
    args = training_args, # hyperparameter defined before
    train_dataset = ds["train"], 
    eval_dataset = ds["test"],
    compute_metrics = compute_accuracy, # evaluation function defined before
    data_collator = data_collator
)

Using cuda_amp half precision backend


# TRAINING AND SAVING THE MODEL

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6090
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 16
  Total optimization steps = 141
  Number of trainable parameters = 335143938
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is

Epoch,Training Loss,Validation Loss


In [ ]:
import os
# Set the output directory
output_dir = '/content/output/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model and tokenizer to the output directory
trainer.save_model(output_dir)
tokz.save_pretrained(output_dir)
